In [ ]:
def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [1]:
import three_desc_model as exponential_new
import numpy as np
import pandas as pd
from cutoffs import Polynomial
from scipy.optimize import minimize,fmin
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import GPy
from IPython.display import display
dataset_full = pd.read_csv('all_dataset.csv', index_col=0)
symbol="Siemens"
#dataset_pos=np.empty(shape=(0, 2))
#dataset_fp=np.empty(shape=(0, 3))
#dataset_ws=np.empty(shape=(0, 1))
k=0
count=0
dataset_full["IDnum"]=pd.Series()
for i in range(len(dataset_full)):
    dataset_full.at[i,"IDnum"]=k
    count = count +1
    if (count==dataset_full["Num_tot_turb"].iloc[i]):
        k=k+1
        count=0
dataset_full["Num_tot_turb"]=dataset_full["Num_tot_turb"].astype(int)
dataset_full["Turb_num"]=dataset_full["Turb_num"].astype(int)
dataset_full["IDnum"]=dataset_full["IDnum"].astype(int)
numsims=dataset_full["IDnum"].iloc[-1]+1
turb = [symbol]
nl=exponential_new.NeighborlistCalculator(cutoff=4001,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)



In [2]:
def f(x):
    Gs = {"Siemens": [{"type":"G2", "turbine":"Siemens","eta":x[0], "offset": 1.},
                  {"type":"G4", "elements":"Siemens","eta":x[1], "gamma":4., "zeta": 1.0},
                  {"type":"G6", "elements":"Siemens","eta":x[2], "gamma": 0.5, "zeta": 1.0}
                 ]}
    finpr=exponential_new.FingerprintCalculator(cutoff=4001,Gs=Gs,Rct=3000,delta_R=100,cone_grad=0.12582561117875557, cone_offset=72.24947126849844)
    count=0
    dataset_fp=np.empty(shape=(0, 3))
    dataset_ws=np.empty(shape=(0, 1))
    for i in range(numsims):
        numturb=dataset_full["Num_tot_turb"].iloc[count]
        position = np.empty((numturb,2))
        ws = np.empty((numturb,1))
        fp = np.empty((numturb,3))
        for k in range(numturb):
            position[k,0]=dataset_full.at[count,"X_coord"]
            position[k,1]=dataset_full.at[count,"Y_coord"]
            ws[k,0]=dataset_full.at[count,"Ref_wind"]
            count = count+ 1
        neigh=nl.calculate(turb*numturb,position)
        fingerprints=finpr.calculate(turb*numturb,position,neigh,symbol)
        fingerprints=np.array(fingerprints)
        #dataset_pos=np.append(dataset_pos,position,axis=0)
        dataset_fp=np.append(dataset_fp,fingerprints,axis=0)
        dataset_ws=np.append(dataset_ws,ws,axis=0)
    dataset=np.concatenate((dataset_fp, dataset_ws),axis=1)
    dataset = pd.DataFrame(dataset, columns = ['Fingerprint(G2)','Fingerprint(G4)','Fingerprint(G6)','Ref_Wind_Speed'])
    X=dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)"]].to_numpy()
    Y=dataset[["Ref_Wind_Speed"]].to_numpy()
    train_dataset = dataset.sample(frac=0.8, random_state=0)
    test_dataset = dataset.drop(train_dataset.index)
    Xtrain=train_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)"]].to_numpy()
    Ytrain=train_dataset[["Ref_Wind_Speed"]].to_numpy()
    Xtest=test_dataset[["Fingerprint(G2)","Fingerprint(G4)","Fingerprint(G6)"]].to_numpy()
    Ytest=test_dataset[["Ref_Wind_Speed"]].to_numpy()
    # define kernel
    ker = GPy.kern.RBF(3,lengthscale=0.1)# + GPy.kern.White(2)
    # create simple GP model
    m = GPy.models.GPRegression(Xtrain,Ytrain,ker)
    # optimize and plot
    m.optimize(max_f_eval = 1000)

    [Ypred,Yvar]=m.predict(Xtest)
    return np.max(Ypred-Ytest)

In [4]:
i=np.linspace(4,6,6)

for k in i:
    x0 = [k, 5., 2.]
    print(f(x0))



0.7406359465508672
0.7683589299014386
0.7937758348131041
0.8159054523169544
0.8354772390848382
0.8535167389025275


In [5]:
x0 = [5.3, 5., 2.]
minimum = fmin(f, x0,disp=True)

Optimization terminated successfully.
         Current function value: 0.262793
         Iterations: 148
         Function evaluations: 266


In [6]:
minimum

array([7.83091952, 1.91278235, 0.8768927 ])

In [ ]:
x0 = [5.3, 5., 2.]
bounds= ((0,10),(0,10),(0,10))
res = minimize(f, x0, bounds= bounds, options={'disp':True})

In [ ]:
count=0
for i in range(numsims):
#    if dataset.loc[dataset['Number of Turbine in the Simulation'] == 2].reset_index(drop=True)
    numturb=dataset["Num_tot_turb"].iloc[count]
    print(count)
    position = np.empty((numturb,2))
    ws = np.empty((numturb,1))
    fp = np.empty((numturb,3))
    for k in range(numturb):
        position[k,0]=dataset.at[count,"X_coord"]
        position[k,1]=dataset.at[count,"Y_coord"]
        ws[k,0]=dataset.at[count,"Ref_wind"]
        count = count+ 1
    #print(position)
    neigh=nl.calculate(turb*numturb,position)
    fp=finpr.calculate(turb*numturb,position,neigh,symbol)
    fp=np.array(fp)
    print(fp)
    dataset_pos=np.append(dataset_pos,position,axis=0)
    dataset_fp=np.append(dataset_fp,fp,axis=0)
    dataset_ws=np.append(dataset_ws,ws,axis=0)
dataset=np.concatenate((dataset_fp, dataset_ws),axis=1)